Section 1: Load Modules

In [1]:
%matplotlib ipympl
import h5py, tifffile
from pathlib import Path
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import xanes_math as xm
import xanes_analysis as xa
xa = reload(xa)
plt.rcParams['figure.figsize'] = [14, 20]

import scyjava_config
scyjava_config.add_options('-Xmx20g')
import imagej
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()
print('\033[04m\033[01m\033[34m     Section 1 finished     ')

Added 387 JARs to the Java classpath.
     Section 1 finished     


Section 2: User Setting Inputs

In [86]:
# file location
aligned_4D_recon_file = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_final.h5"
raw_file_template = "/media/xiao_usb/2019_03/FengLin/fly_scan_id_{0}.h5"
out_fn = os.path.join(str(Path(recon_path_template).parents[1]), 
                      '3D_XANES_scan_id_'+str(scan_id_s)+'-'+str(scan_id_e)+'.h5')

# if it is whiteline, set is_wl = True, otherwise is_wl = False
is_wl = True
# scan_id start and end
scan_id_s = 12450
scan_id_e = 12466

# if you want to make a mask with a recon, set make_mask = True
make_mask = True
# scan id for making mask; it is good to choose one after the edge
mask_scan_id = 12460
# set threshold for making the mask; you can pre-test out the threshold in imagej
mask_threshold = 3e-3

# edge_offset from Co
edge_offset_2_Co = 7.709 - 7.709
# estimated edge energy
edge_eng = 7.709 + edge_offset_2_Co
# end poit of the pre-edge relative to the edge_eng in keV
pre_ee = -0.05
# start point of the post-edge relative to the edge_eng in keV
post_es = 0.1
# how many times of the edge jump magnitude should be compared to the pre-edge standard deviation
edge_jump_threshold = 4
# how much should the pre_edge be offset up for validating if the post edge trend is in a reasonable range
# this is a factor to the pre-edge deviation
pre_edge_threshold = 3
# define an energy range for 0.5 absorption postion fitting
ep_eng_s = 7.716 + edge_offset_2_Co
ep_eng_e = 7.731 + edge_offset_2_Co
# define an energy range for whiteline peak postion fitting
wl_eng_s = 7.725 + edge_offset_2_Co +0.001
wl_eng_e = 7.735 + edge_offset_2_Co - 0.003

# define an energy range for edge_pos display
ep_vmin = 7.720 + edge_offset_2_Co
ep_vmax = 7.731 + edge_offset_2_Co
# define an energy range for whiteline display
wl_vmin = 7.7285 + edge_offset_2_Co
wl_vmax = 7.7305 + edge_offset_2_Co
# define path and file name to save xanes analysis results; if you use the default path and name as below,
# you don't need to change anything. otherwise, give your full path and file name below.
#out_fn = os.path.join(str(Path(fn_template).parent), 'xanes_analysis_' + str(Path(fn_template).name)).format(scan_id)
#print(out_fn)
print('\033[04m\033[01m\033[34m     Section 2 finished     ')

     Section 2 finished     


Section 3: Read data

In [87]:
fn = []
eng = []
for ii in range(scan_id_s, scan_id_e):
#scan_id = 12455
    fn.append(sorted(list(Path(recon_path_template.format(ii)).glob('*.*'))))
    f = h5py.File(raw_file_template.format(ii), 'r')
    eng.append(np.array(f['X_eng']))
    f.close()

fn = np.array(fn)
eng = np.array(eng)
print(fn.shape, eng.shape)
print(eng)

ny, nx = tifffile.imread(str(fn[0, 0])).shape
imgs = np.ndarray([fn.shape[0], ny, nx])

if make_mask:
    mask = np.ndarray([fn.shape[1], ny, nx])
    
if is_wl:
    xanes3d = np.ndarray([fn.shape[1], ny, nx])
else:
    xanes3d = np.ndarray([3, fn.shape[1], ny, nx])

xana = xa.xanes_analysis(imgs, eng, edge_eng, pre_ee=pre_ee, post_es=post_es, 
                         edge_jump_threshold=edge_jump_threshold, pre_edge_threshold=pre_edge_threshold)

print('Your results will be saved in the file', '\"'+out_fn+'\"')
print('\033[04m\033[01m\033[34m     Section 3 finished     ')

(16, 130) (16,)
[7.7199995  7.72100261 7.72199893 7.72300259 7.72399944 7.72500363
 7.72600102 7.72699868 7.7279966  7.72900187 7.73000032 7.73099905
 7.73199804 7.7329973  7.73399682 7.73499662]
Your results will be saved in the file "/media/xiao_usb/2019_03/FengLin/3D_XANES_scan_id_12450-12466.h5"
     Section 3 finished     


Section 4: Analyze 3D XANES

In [88]:
if make_mask:
    idx = xm.index_of(np.arange(scan_id_s, scan_id_e), mask_scan_id)
    for ii in range(fn.shape[1]):
        imgs[idx] = tifffile.imread(str(fn[idx, ii]))    
        mask[ii] = (imgs[idx]>mask_threshold).astype(np.uint8)[:]  
        
for ii in range(fn.shape[1]):
    for jj in range(fn.shape[0]):
        imgs[jj] = tifffile.imread(str(fn[jj, ii]))
    
    xana.spectrum[:] = imgs[:]
    print(xana.spectrum.shape)
    if is_wl:
        xana.fit_whiteline(wl_eng_s, wl_eng_e)
        xanes3d[ii] = xana.whiteline_pos[:]*mask[ii]
    else:
        xana.edge_jump()
        xana.cal_pre_edge_sd()
        xana.cal_post_edge_sd()
        xana.fit_pre_edge()
        xana.fit_post_edge()
        # functions work but still need to improve member variable archietecture design
        xana.create_edge_jump_filter(edge_jump_threshold)
        xana.create_fitted_edge_filter(pre_edge_threshold)
        xana.normalize_xanes(edge_eng, order=0)
        xana.fit_edge_pos(ep_eng_s, ep_eng_e)
        xana.fit_whiteline(wl_eng_s, wl_eng_e)
        xanes3d[0, ii] = xana.normalized_spectrum[:]*mask[ii]
        xanes3d[1, ii] = xana.edge_eng_pos[:]*mask[ii]
        xanes3d[2, ii] = xana.whiteline_pos[:]*mask[ii]                
    print(ii)

if is_wl:
    xana.save_results(out_fn, dtype='3D_XANES', **{'whiteline': xanes3d})
else:
    xana.save_results(out_fn, dtype='3D_XANES', **{'normalized_spectrum': xanes3d[0, :], 
                                                   'endg_eng_pos': xanes3d[1, :],
                                                   'whiteline_pos': xanes3d[2, :],
                                                   'mask': mask})

print('Your results will be saved in the file', '\"'+out_fn+'\"')
print('\033[04m\033[01m\033[34m     Section 4 finished     ')   

(16, 185, 198)
0
(16, 185, 198)
1
(16, 185, 198)
2
(16, 185, 198)
3
(16, 185, 198)
4
(16, 185, 198)
5
(16, 185, 198)
6
(16, 185, 198)
7
(16, 185, 198)
8
(16, 185, 198)
9
(16, 185, 198)
10
(16, 185, 198)
11
(16, 185, 198)
12
(16, 185, 198)
13
(16, 185, 198)
14
(16, 185, 198)
15
(16, 185, 198)
16
(16, 185, 198)
17
(16, 185, 198)
18
(16, 185, 198)
19
(16, 185, 198)
20
(16, 185, 198)
21
(16, 185, 198)
22
(16, 185, 198)
23
(16, 185, 198)
24
(16, 185, 198)
25
(16, 185, 198)
26
(16, 185, 198)
27
(16, 185, 198)
28
(16, 185, 198)
29
(16, 185, 198)
30
(16, 185, 198)
31
(16, 185, 198)
32
(16, 185, 198)
33
(16, 185, 198)
34
(16, 185, 198)
35
(16, 185, 198)
36
(16, 185, 198)
37
(16, 185, 198)
38
(16, 185, 198)
39
(16, 185, 198)
40
(16, 185, 198)
41
(16, 185, 198)
42
(16, 185, 198)
43
(16, 185, 198)
44
(16, 185, 198)
45
(16, 185, 198)
46
(16, 185, 198)
47
(16, 185, 198)
48
(16, 185, 198)
49
(16, 185, 198)
50
(16, 185, 198)
51
(16, 185, 198)
52
(16, 185, 198)
53
(16, 185, 198)
54
(16, 185, 198)
55
(1